# 03b - Vertex AI + BQML - Online Predictions with BQML Models

Models built with BigQuery ML (BQML), like the one in (03a), can also be exported for use and deployment outside of BigQuery.  A Vertex AI Endpoint can be used for online predictions with an exported model.  This demonstration shows the process of exporting and deploying a BQML model with Vertex AI.

### Video Walkthrough of this notebook:
Includes conversational walkthrough and more explanatory information than the notebook:

<p><center><a href="https://youtu.be/7y_t_bW0LHQ" target="_blank" rel="noopener noreferrer"><img src="architectures/thumbnails/playbutton/03b.png" width="50%"></a></center></p>

### Prerequisites:
-  03a - BigQuery Machine Learning (BQML) - Machine Learning with SQL

### Overview:
-  Export the BigQuery ML model built in (03a) to a GCS bucket URI
   -  EXPORT MODEL …
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict

### Resources:
-  [Export formats for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/exporting-models)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/03b_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/03b_console.png">

---
## Setup

inputs:

In [12]:
REGION = 'us-central1'
PROJECT_ID='ma-mx-presales-lab'
DATANAME = 'fraud'
NOTEBOOK = '03b'

# Resources
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [13]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [14]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [15]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = "vertex-ai-mlops-bucket"
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
params = {"URI": URI}
DIR = f"temp/{NOTEBOOK}"

environment:

In [16]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [17]:
export = bigquery.query(query = f"EXPORT MODEL {DATANAME}.{DATANAME}_lr OPTIONS(URI = '{URI}')")

In [18]:
export.result()

---
## Serving

### Upload The Model
https://googleapis.dev/python/aiplatform/latest/aiplatform.html?highlight=aiplatform%20model%20upload#google.cloud.aiplatform.Model.upload

In [19]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/825075454589/locations/us-central1/models/7453057161065660416/operations/3364561175051239424
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/825075454589/locations/us-central1/models/7453057161065660416
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/825075454589/locations/us-central1/models/7453057161065660416')


In [20]:
model.display_name

'03b_fraud_20220330162620'

### Create An Endpoint

In [21]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/825075454589/locations/us-central1/endpoints/5318865509133844480/operations/4641331669410775040
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/825075454589/locations/us-central1/endpoints/5318865509133844480
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/825075454589/locations/us-central1/endpoints/5318865509133844480')


In [22]:
endpoint.display_name

'03b_fraud_20220330162620'

### Deploy Model To Endpoint

In [23]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/825075454589/locations/us-central1/models/7453057161065660416 to Endpoint : projects/825075454589/locations/us-central1/endpoints/5318865509133844480
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/825075454589/locations/us-central1/endpoints/5318865509133844480/operations/6129771346256723968
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/825075454589/locations/us-central1/endpoints/5318865509133844480


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [24]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [25]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,6126,-0.130702,1.026568,2.692793,2.659141,0.764928,1.800330,0.317986,-0.392976,0.851460,...,-0.370958,-1.058635,-0.498915,0.054906,-0.590204,-0.537678,0.0,0,70526fff-bdf3-459f-bbc6-a18e8133bb35,TEST
1,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0,e8b3c609-62a9-4ec3-9ff8-fd0974b07602,TEST
2,35599,1.168909,-0.139981,-0.095518,1.735426,1.699903,4.646212,-1.191502,1.205700,0.333882,...,-0.163863,1.045593,0.775343,0.228745,0.050846,0.020019,0.0,0,c5ac2c63-c26b-492e-bc8e-5f98e92ec866,TEST
3,49561,1.333331,-0.845997,1.161578,-0.610965,-1.635783,-0.198304,-1.331531,0.212857,-0.208834,...,-0.095332,0.367347,0.398274,-0.059810,0.041674,0.011653,0.0,0,aca44b9b-e816-4767-b636-0356637c286b,TEST


In [26]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET,'splits'])]].to_dict(orient='records')[0]
#newob

In [27]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [28]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[{'Class_values': ['1', '0'], 'Class_probs': [0.1841480504765189, 0.8158519495234812], 'predicted_Class': ['0']}], deployed_model_id='3618598320127082496', explanations=None)

In [29]:
prediction.predictions[0]#['classes'][np.argmax(prediction.predictions[0]['scores'])]

{'Class_values': ['1', '0'],
 'Class_probs': [0.1841480504765189, 0.8158519495234812],
 'predicted_Class': ['0']}

In [30]:
prediction.predictions[0][f'{VAR_TARGET}_values'][np.argmax(prediction.predictions[0][f'{VAR_TARGET}_probs'])]

'0'

### Get Predictions: REST

In [20]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [21]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    {
      "Class_probs": [
        0.0990060111289882,
        0.90099398887101179
      ],
      "predicted_Class": [
        "0"
      ],
      "Class_values": [
        "1",
        "0"
      ]
    }
  ],
  "deployedModelId": "5358623849594224640",
  "model": "projects/764015827198/locations/us-central1/models/6040246292024655872",
  "modelDisplayName": "03b_fraud_20211129141713"
}


### Get Predictions: gcloud (CLI)

In [22]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'Class_probs': [0.0990060111289882, 0.9009939888710118], 'Class_values': ['1', '0'], 'predicted_Class': ['0']}]


---
## Remove Resources
see notebook "99 - Cleanup"